In [1]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import datetime
from datetime import timedelta 
import pandas as pd
import pickle
import os

In [2]:
def get_tweet_age(tweet_time_str):
    tweet_time = datetime.strptime(tweet_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    now = datetime.utcnow()
    age_minutes = (now - tweet_time).total_seconds() / 60
    return age_minutes

In [3]:
def should_scrape_tweet(tweet_age, comment_count):
    if tweet_age <= 5 and comment_count > 5:
        return True
    elif tweet_age <= 60 and comment_count > 10:
        return True
    elif comment_count > 20:
        return True
    return False

In [3]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # optional

# DO NOT add --headless

driver = webdriver.Chrome(options=chrome_options)
driver.get("https://google.com")
time.sleep(3)
#input("Press Enter to close...")  # to keep it open
driver.quit()


In [4]:
def scrape_twitter():
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)

    # Login manually
    driver.get("https://x.com/i/flow/login")
    print("Login manually within 30 seconds...")
    time.sleep(30)

    search_term = "Bitcoin"
    search_url = f"https://x.com/search?q=%23{search_term}&src=typed_query&f=live"
    driver.get(search_url)
    time.sleep(5)

    tweets_seen = set()
    scraped_data = []
    #save_path = "C:/crypto_project/twitter_data_demo.txt"
    #save_path = "/Users/payeldutta/Desktop/My File/twitter_data.txt"
    save_path = "C:/crypto_project/twitter_data_demo.csv"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)


    for _ in range(5):  # Smaller scroll range for stability
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        tweets = driver.find_elements("css selector", "a[href*='/status/']")
        tweet_links = list({tweet.get_attribute("href") for tweet in tweets if '/status' in tweet.get_attribute("href")})

        for link in tweet_links:
            if link in tweets_seen:
                continue
            tweets_seen.add(link)

            driver.get(link)
            time.sleep(4)

            try:
                
                tweet_time_str = driver.find_element("tag name", "time").get_attribute("datetime")
                tweet_age = get_tweet_age(tweet_time_str)
            except:
                continue  # Skip if no timestamp

            # Count number of comments
            texts = driver.find_elements('css selector', "div[data-testid='tweetText']")
            comment_count = len(texts) - 1  # First is main tweet

            # Apply your conditions
            if should_scrape_tweet(tweet_age, comment_count):
                for idx, t in enumerate(texts):
                    scraped_data.append({
                        "tweet_type":"main" if idx == 0 else "comment",
                        "tweet_text": t.text,
                        "timestamp":tweet_time_str,
                        "coin": f"{search_term}"
                    })
                '''
                with open(save_path, "a", encoding="utf-8") as f:
                    for idx, t in enumerate(texts):
                        if idx == 0:
                            f.write(f"\nMain Tweet @ {tweet_time_str}:\n{t.text}\n")
                        else:
                            f.write(f"↳ Comment @ {tweet_time_str}:\n{t.text}\n")
                '''
        # Back to search page for next scroll
        driver.get(search_url)
        time.sleep(3)
    df = pd.DataFrame(scraped_data)
    df.to_csv(save_path, index = False, encoding = 'utf-8-sig')
    print(f"Scraping complete.{len(scraped_data)} Rows saved to csv")
    driver.quit()


In [5]:
scrape_twitter()

Login manually within 30 seconds...


KeyboardInterrupt: 

In [ ]:
def scrape_twitter_with_cookies():
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)

    # Load domain first
    driver.get("https://x.com")
    time.sleep(3)

    # Login 
    driver.get("https://x.com/i/flow/login")
    print("login manually")
    time.sleep(30)
    
    driver.get("https://x.com/home")
    time.sleep(5)

    # Search by coin name
    search_term = "Bitcoin"
    search_url = f"https://x.com/search?q=%23{search_term}&src=typed_query"
    driver.get(search_url)
    time.sleep(5)

    # Scroll page
    for _ in range(2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    # Scrape tweets
    tweets = driver.find_elements("css selector", "div[data-testid='tweetText']")
    tweet_texts = [tweet.text for tweet in tweets]

    # Save tweets
    save_path_2 = "C:/Users/Payal Dutta/Documents/twitter_data.txt"
    with open(save_path_2, "w", encoding="utf-8") as f:
        for t in tweet_texts:
            f.write(t + "\n")
    print(f"Collected {len(tweet_texts)} tweets")

    # Logout (optional)
    driver.get("https://x.com/logout")
    time.sleep(2)
    try:
        logout_button = driver.find_element("css selector", "div[role='button'][data-testid='confirmationSheetConfirm']")
        logout_button.click()
        print("Logged out successfully")
    except:
        print("Logout failed or already logged out.")

    driver.quit()
